In [ ]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, chi2


In [4]:
# Load the dataset
df = pd.read_csv('/content/Hinglish.csv', encoding='latin-1')

In [5]:
# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Extract target variable from the train dataset
y_train = train_df['label']

In [7]:
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Tokenize the tweets and get the BERT embeddings
def get_bert_embeddings(text):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    tokens_tensor = torch.tensor([tokens])
    with torch.no_grad():
        outputs = bert_model(tokens_tensor)
        last_hidden_state = outputs[0]
        embedding = torch.mean(last_hidden_state, dim=1).numpy()
    return embedding


In [9]:
# Get the BERT embeddings for train and test sets
train_embeddings = np.vstack(train_df['tweet'].apply(get_bert_embeddings))
test_embeddings = np.vstack(test_df['tweet'].apply(get_bert_embeddings))

In [10]:
# Use TF-IDF vectorizer for the machine learning algorithms
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['tweet'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['tweet'])

In [11]:
# Feature selection using Chi-Square
selector = SelectKBest(chi2, k=min(500, X_train_tfidf.shape[1]))
selector.fit(X_train_tfidf, y_train)
X_train_tfidf = selector.transform(X_train_tfidf)
X_test_tfidf = selector.transform(X_test_tfidf)

In [12]:
# Create individual classifiers
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
nb = MultinomialNB()
ridge = RidgeClassifier()
gb = GradientBoostingClassifier()
lr = LogisticRegression()
#X_train_tfidf = X_train_tfidf.toarray()
#X_test_tfidf = X_test_tfidf.toarray()
#lda = LinearDiscriminantAnalysis()

In [13]:
# Create a list of classifiers
classifiers = [('ada', ada), ('nb', nb), ('ridge', ridge), ('gb', gb),('lr', lr)]

In [14]:
# Create the ensemble model
ensemble = VotingClassifier(classifiers, voting='hard')

In [15]:
# Fit the ensemble model on the training data
ensemble.fit(X_train_tfidf, y_train)

VotingClassifier(estimators=[('ada',
                              AdaBoostClassifier(n_estimators=100,
                                                 random_state=0)),
                             ('nb', MultinomialNB()),
                             ('ridge', RidgeClassifier()),
                             ('gb', GradientBoostingClassifier()),
                             ('lr', LogisticRegression())])

In [16]:
# Predict the test data using the ensemble model
y_test = test_df['label']
y_pred_ensemble = ensemble.predict(X_test_tfidf)

In [17]:
# Print the performance metrics of the ensemble model
print("Ensemble Model")
print("Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Precision:", precision_score(y_test, y_pred_ensemble, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_ensemble, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred_ensemble, average='weighted'))

Ensemble Model
Accuracy: 0.6888646288209607
Precision: 0.6903374967504697
Recall: 0.6888646288209607
F1-Score: 0.6242865088396807


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score, cohen_kappa_score, matthews_corrcoef

In [19]:
print("Kappa:", cohen_kappa_score(y_test, y_pred_ensemble))
print("MCC:", matthews_corrcoef(y_test, y_pred_ensemble))

Kappa: 0.1714695577108618
MCC: 0.23308523778359105


In [20]:
print("AUC: ", round(roc_auc_score(y_test, y_pred_ensemble), 3))


AUC:  0.571
